In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline
from sklearn.model_selection import train_test_split
import dask.dataframe as dd # Install dask library for code to import

In [5]:
parkingDD = dd.read_csv('/Users/Omar/Desktop/Info 370/nyc-parking-violations/data/Parking_Violations_Issued_-_Fiscal_Year_2018.csv')

## Deleting and dropping unused columns and NA values 

In [79]:
def initialClean(df):
    drop_columns = [
 'Plate ID',
 'Street Code1',
 'Street Code2',
 'Street Code3',
 'Vehicle Expiration Date',
 'Violation Location',
 'Violation Precinct',
 'Issuer Precinct',
 'Issuer Code',
 'Issuer Command',
 'Issuer Squad',
 'Time First Observed',
 'Violation In Front Of Or Opposite',
 'House Number',
 'Intersecting Street',
 'Date First Observed',
 'Violation Legal Code',
 'Days Parking In Effect    ',
 'From Hours In Effect',
 'To Hours In Effect',
 'Vehicle Color',
 'Unregistered Vehicle?',
 'Meter Number',
 'Violation Post Code',
 'Violation Description',
 'No Standing or Stopping Violation',
 'Hydrant Violation',
 'Double Parking Violation']
    df = df.drop(drop_columns, axis=1)
    df = df.dropna(how = 'any')
    df = df.reset_index(drop=True)
    return df

In [80]:
parkingDD = initialClean(parkingDD)

## Finding indices of colors which have a gibberish code and deleting those rows 

In [81]:
def colorCorrection(df):
    remove = ['PRW','MH','TPE','FT','TAUPE','GRU','XXX','WHBR','AZ','PL','GM','LGITH','BLPR','BK SL','RDBR','LEXUS','RDPR','BRK','MAXIM','BRT','GRBK','GU','BT','BL/SI','ZIWIY','WT BK','BN.','LTBK', 'PRBL','BLTN','VAN','BLT','BRG','GY TN','BLYW','BLBR','GY/BL','DODGE','MAZ','BWTH','IW','NMI','NOCLR','OTH','WYBLU','BKPR','GKN','WAJP','GYG','BRY','KX','VN  R','DKTN','RDTN','TQ','PLE','ORBK','GLGR','YAN','QGY','MITI','BL/WH','QUEEN','GRT','BLK/S','ORWH','BLE','MULT','ZJHT','BWR','BBRW','GRG','TN.','DARL','GYMR','GRGL','CNH', 'SC','GLBK', 'VN  S', 'COM''WT BL','LTPR','BLGL','GRAT','WORKH','GRBL','CRM','MUILT','K.','VLK','GA','RD BK','GYRD','ZKCXN','YE/OR','RDWH','GLK','GT','MC','BLR','KD','GRTN''WHOR','BM','WJT','BK/GR','DI', 'BKT', 'SIM','RDGR', 'RDT', 'WH BL','BE','RUST','WHBK','RY','UJ','GYGL','NH','WH/BL','RDBK','TL', 'BKRD','SR','BRU','BUS','GYQ','MUL','DECAL','BLBK','RDG','BON','LTBR','LTTN','TNGY','TNBL','GRGY','B L','BLWH','BK BL','TNG','RDGY','HRAU','CHAMP','BLKQ','BJ','WB','GYTN','BKBL','BLKL','WHRD','BRGR','BKG','BGE','-','TNRD','GRW','NOCL','W/B','GLBL','BRTOW','NO','RDBL',', ,U','BRZ','UNKNO','BLW','BLGR','GYBL','BLRD','WN','WHGR','GYGR','RDMR','BLB','WHG','GYBK','WHO','BKGR','TNR','OR BK','BLG','NOC','BKTN','MULTI','GYGY','RDW','WT/BL','BLGY','GYB','GYBR','UNK','BKGY', 'DK/', 'TN', 'PR', 'ALUMI', 'OTHER', 'LT/', 'WHB', 'WHBL', 'WHGY', 'TAN', 'LTT']
    
    color = ''
    
    indices = []
    
    
    for index,row in df.iterrows():
        if row['Vehicle Color'] in remove:
            indices.append(index)
    return indices

In [82]:
indices = colorCorrection(parkingDD)

## Converting the Dask dataframe back into Pandas

In [83]:
def convert(df):
    #parkingPD = df.compute()
    df.drop(df.index[indices], inplace=True)
    df = df.reset_index(drop=True)
    return df

In [84]:
parkingPD = convert(parkingDD)

## Renaming colors across the various typos

In [85]:
def colorRenaming(df):
    black = ['BKBK','BLBL','BLLK','B K','BLACJ','BLKI','BL AC','BLC','CHAR','BLACL','BKL','BLIC','B LK','BLK.','VLACK','B LAC''BIACK','BLCK','BLACC','Black','BLAVK','BIK','BK,','BL/','BLA','BLAC','BK/','BKACK','BLAKC','BK', 'BLACK', 'BLK']
    white = ['WITE','WHOTE','WHICH','WIT','CREAM','WHLE','WHWH','WHIYE','ITE',"WH'",'WHITW','WJOTE','WHTN','WHITR','WHT/S','WHTE','WT.','PEARL','WTE','W','WH.','WHIT', 'WHITE', 'WH', 'WT', 'WH/', 'WHT', 'WHI']
    silver = ['SIV','Silve','SIL.','SVR','SI','SLIVE','SIVL','SILO','SLVER','SILGV','SILVQ','SV','SIVLE','SILER','SLR','SL.','SILVE', 'SIL', 'SL', 'SILV', 'SILVR']
    green = ['GRGR','Green','DK GN','GR.','LGR','LT/GR','DK GR','GN.','DKGR','G','GRE','GREEB','LTGR','GN','GREEN', 'GR', 'GRN', 'GREN', 'GREE']
    grey = ['GEAY','GEY','GARY','DGRAY','GRRAY','GREY.','GHRAY','GRY.','GR/','GRAY.','GRA','HREY','GAEY','GY/','Gray','GY', 'GREY', 'GRAY', 'GRY', 'LTGY', 'LTG', 'DKGY', 'DKG', 'GY.']
    red = ['RR','RED.','RD.','RD/','R','DKRD','RD', 'RED', 'DKR']
    maroon = ['MROO','MRN','MN','DKMR','DKM','MAROO','MAR', 'MR', 'MARO']
    gold = ['GDL','GNY','GLOD','GD.','LTGL','GLTN','GOLD.','GL.','GD','GOLD', 'GL', 'GLD']
    blue = ['TEAL','BLUEW','BLIE','NAVY','B LUE','BL UE','BLUE.','BLUED','LBL','BI','NAVY.','DBL','LBUE','LB','LTBL','DKB','B','LTB', 'BL.', 'DKBL', 'BL', 'BLUE', 'BLU']
    brown = ['BRBR','BROWY','BROK','BRONK','BRZ', 'BRONZ','COPPE','DKBR','BWN','BRW','BROW','BRO','BROWN', 'BR', 'BRN', 'BRWN', 'BW']
    orange = ['O','ORAGE','OG','ORANE','DKOR','ORNGE','ORAG','ORA','OR.','0','ORN','ORAN','ONG','ORANG', 'OR', 'ORG']
    yellow = ['YELO','YLL','YEDLL','LTYW','YELW','YLW','YELLW','YL','YEL','YELL','Y','YELLO', 'YW']
    purple = ['LAVEN','DKPR','PUPLE','TNGL','TURPL','PURP','PURPL', 'PUR']
    pink = ['LTPK','DKPK','LTP','PINK']
    burgundy = ['BY','BUR','BURGY','BARG','BURGE','BERG','BU','BGDY','BURGA','BRGY','BN','BURGU', 'BURG']
    beige = ['BEGE','BGE.','BIGE','BAGE','BEIG','BEIGE', 'BEIEG', 'BIEGE', 'BG']
    
    color = ''
    
    for index, row in df.iterrows():
        if row['Vehicle Color'] in black:
            color = 'black'
        elif row['Vehicle Color'] in white:
            color = 'white'
        elif row['Vehicle Color'] in silver:
            color = 'silver'
        elif row['Vehicle Color'] in green:
            color = 'green'
        elif row['Vehicle Color'] in grey:
            color = 'grey'
        elif row['Vehicle Color'] in red:
            color = 'red'
        elif row['Vehicle Color'] in maroon:
            color = 'maroon'
        elif row['Vehicle Color'] in gold:
            color = 'gold'
        elif row['Vehicle Color'] in blue:
            color = 'blue'
        elif row['Vehicle Color'] in brown:
            color = 'brown'
        elif row['Vehicle Color'] in orange:
            color = 'orange'
        elif row['Vehicle Color'] in yellow:
            color = 'yellow'
        elif row['Vehicle Color'] in purple:
            color = 'purple'
        elif row['Vehicle Color'] in pink:
            color = 'pink'
        elif row['Vehicle Color'] in burgundy:
            color = 'burgundy'
        elif row['Vehicle Color'] in beige:
            color = 'beige'
        df.at[index,'Vehicle Color'] = color
    return df

In [86]:
parkingPD = colorRenaming(parkingPD)

## Removing Vehicle Body Types which do not follow city codes

In [87]:
def vehicleBodyTypes(df):
        bodyTypes = ['FIRE', 'CONV', 'SEDN', 'SUBN', '4DSD', '2DSD', 'H/WH', 'ATV', 'MCY', 'H/IN', 'LOCO', 'CUST', 'RPLC', 'AMBU', 'P/SH', 'RBM', 'R/RD', 'RD/S', 'S/SP', 'SN/P', 'TRAV', 'MOBL', 'TR/E', 'T/CR', 'TR/C', 'SWT', 'W/DR', 'W/SR', 'FPM', 'MCC', 'EMVR', 'TRAC', 'DELV', 'DUMP', 'FLAT', 'PICK', 'STAK', 'TANK', 'REFG', 'TOW', 'VAN', 'UTIL', 'POLE', 'BOAT', 'H/TR', 'SEMI', 'TRLR', 'LTRL', 'LSVT', 'BUS', 'LIM', 'HRSE', 'TAXI', 'DCOM', 'CMIX', 'MOPD', 'MFH', 'SNOW', 'LSV']
        indices = []
        for index, row in df.iterrows():
            if row['Vehicle Body Type'] not in bodyTypes:
                color = 'black'
                indices.append(index)
        return indices

In [89]:
indices = vehicleBodyTypes(parkingPD)
parkingPD.drop(parkingPD.index[indices], inplace=True)

## Creating testing/training .csv

In [90]:
def writeCSV(df):
    df.to_csv("cleanedData.csv")
    nyc_train_features, nyc_test_features, nyc_train_outcome, nyc_test_outcome = train_test_split(df, df['Violation Code'], test_size=0.30)
    nyc_train_features.to_csv("nyc_train_features.csv")
    nyc_test_features.to_csv("nyc_test_features.csv")
    nyc_train_outcome.to_csv("nyc_train_outcome.csv")
    nyc_test_outcome.to_csv("nyc_train_features.csv")

In [91]:
writeCSV(parkingPD)